In [1]:
import networkx as nx
from regraph import Rule, plot_rule


from regraph.neo4j.hierarchy import Neo4jHierarchy
import regraph.neo4j.cypher_utils as cypher
from regraph.neo4j.rewriting_utils import propagate_down_v2, propagate_up_v3

In [2]:
# initialize the neo4j driver, wrappped into Neo4jHierarchy object
h = Neo4jHierarchy(uri="bolt://localhost:7687", user="neo4j", password="admin")

# Propagation Down

In [ ]:
nodes = [
    ("a", {"name": "Garfield", "age": 2}), 
    ("b", {"name": "Rex", "age": 7}),
    ("c", {"name": "Paul", "age": 45}),
    ("d", {"name": "Joe", "age": 10})
]
edges = [
    ("b", "c", {"type": "pet_of"}),
    ("b", "d", {"type": "pet_of"}), 
    ("a", "b", {"type": "enemies"})
]
h.add_graph('actionGraph', nodes, edges)

In [ ]:
nodes = [
    ("a", {"type": "Cat", "age": 2}), 
    ("b", {"type": "Dog", "age": 7}),
    ("c", {"type": "Adult", "age": 45}),
    ("d", {"type": "Child", "age": 10})
]
edges = [
    ("b", "c", {"type": "pet_of", "number": 2}),
    ("b", "d", {"type": "pet_of", "number": 3}), 
    ("a", "c", {"type": "pet_of", "number": 4}),
    ("a", "d", {"type": "pet_of", "number": 5}), 
    ("a", "b", {"type": "enemies", "number": 1})
]
h.add_graph('metaModel', nodes, edges)

In [ ]:
mapping = {
    "a":"a",
    "b":"b",
    "c":"c",
    "d":"d"
}
h.add_typing('actionGraph', 'metaModel', mapping)

In [ ]:
pattern = nx.DiGraph()
pattern.add_nodes_from(["w", "x", "y", "z"])
pattern.add_edges_from([("x", "y"), ("x", "z"), ("w", "x")])

In [ ]:
rule = Rule.from_transform(pattern)
rule.inject_merge_nodes(["w", "x"])
rule.inject_merge_nodes(["y", "z"])

In [ ]:
ag = h.access_graph('actionGraph')
instances = ag.find_matching(pattern)
print("Instances: ", instances)

In [ ]:
changes = h.rewrite('actionGraph', rule, instances[0])

In [ ]:
h.propagation_down_v3('actionGraph', changes)

## Propagation Up

In [4]:
nodes = [
    ("a", {"name": "Garfield", "age": 2}), 
    ("b", {"name": "Rex", "age": 7})
]
edges = [
    ("a", "b", {"type": ["enemies"]}),
    ("a", "a", {"type": ["likes"]})
]
h.add_graph('actionGraph', nodes, edges)

In [5]:
nodes = [
    ("a", {"type": "Animal", "age": [2, 7]})
]
edges = [
    ("a", "a", {"type": ["enemies", "likes"]})
]
h.add_graph('metaModel', nodes, edges)

In [6]:
mapping = {
    "a":"a",
    "b":"a"
}
h.add_typing('actionGraph', 'metaModel', mapping)

In [7]:
pattern = nx.DiGraph()
pattern.add_nodes_from(["x"])
pattern.add_edges_from([("x", "x")])

In [8]:
rule = Rule.from_transform(pattern)
clone_name1, _ = rule.inject_clone_node("x")
clone_name2, _ = rule.inject_clone_node("x")

In [9]:
ag = h.access_graph('metaModel')
instances = ag.find_matching(pattern)
print("Instances: ", instances)

Instances:  [{'x': 'a'}]


In [10]:
changes = h.rewrite('metaModel', rule, instances[0])

x
x1
x2


In [11]:
h.propagation_up_v3('metaModel')

Rewritting ancestors of metaModel...
-->  actionGraph
Rewritting ancestors of actionGraph...


## Complex Hierarchy

In [ ]:
#h.add_graph("S")
#h.add_graph("A")
#h.add_graph("B")
#h.add_graph("C")
#h.add_graph("D")
#h.add_graph("T")